In [ ]:
!pip -q install mplfinance
!pip -q install --upgrade ta


from IPython.core.display import display, HTML, Javascript
from datetime import datetime
import os
from IPython.core.display import display, HTML
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

import matplotlib.ticker as mtick
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import mplfinance as mpf


from inspect import getmembers
from pprint import pprint
from ta import add_all_ta_features



%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)
plt.style.use('bmh')




warnings.simplefilter(action='ignore', category=FutureWarning)

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)  # Max. number of rows to display    

<br><br>
<div>    
<!--     <div style = "float:left; width:55%; overflow:hidden;">         -->
        <center><img src="https://storage.googleapis.com/kaggle-competitions/kaggle/30894/logos/header.png" style = "max-height:300px;"></center> 
<!--     </div> -->
<!--     <div style = "float:right; width:35%; overflow:hidden;"> -->
<!--         <img src="img/meditation/Meditation3.gif">  -->
<!--     </div> -->
</div>

# Utils

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def search_null(data):
    n=data.isnull().sum().where(lambda _ : _ > 0).dropna()
    df = pd.DataFrame({'columns':n.index, 'count nulls':n.values})
    df["% Total"] = (df["count nulls"] /  data.shape[0]).round(4)*100.00
    return df    

def group_trading_timeseries(df, time_step_size):
    df = df.sort_index()
    df = df.resample('{}'.format(time_step_size)).agg({
            'Count': lambda df: df.sum(),
            'Open': lambda s: s.iloc[0] ,
            'High': lambda df: df.max(), 
            'Low': lambda df: df.min(),
            'Close': lambda df: df.iloc[-1], 
            'Volume': lambda df: df.sum(),
            'VWAP' : lambda df: df.mean(),
            'Target' : lambda df: df.mean()
            }) 
    return df 

def open_resampler(arraylike):
    nrows = arraylike.shape[0]
    if nrows == 0:
        return np.nan
    else:
        return arraylike.iloc[0]

def close_resampler(arraylike):
    nrows = arraylike.shape[0]
    if nrows == 0:
        return np.nan
    else:
        return arraylike.iloc[-1]

def group_trading_timeseries_all(df, time_step_size):
    df = df.sort_values(["Asset_ID", "timestamp"], ascending = (False, True))
    df = df.groupby("Asset_ID").resample('{}'.format(time_step_size)).agg({
            'Count': lambda df: df.sum(),
            'Open': open_resampler,
            'High': lambda df: df.max(), 
            'Low': lambda df: df.min(),
             'Close': close_resampler,
            'Volume': lambda df: df.sum(),
            'VWAP' : lambda df: df.mean(),
            'Target' : lambda df: df.mean()
            }) 
    return df 

def update_indicator_property(indicators, name, prop, value):
    for indicator in indicators:
        if indicator.name == name:
            indicator.__setattr__(prop, value)
            break

def filter_indicators(indicators, filter_list=[], filter_wildcard="*"):
    if filter_wildcard != "*" and len(filter_list) ==0:
        return [indicator for indicator in indicators if (indicator.name.startswith(filter_wildcard) and (indicator.is_support_indicator == False) )   ]
    elif filter_wildcard != "*" and len(filter_list) >0:
        return [indicator for indicator in indicators if (indicator.name.startswith(filter_wildcard) and (indicator.is_support_indicator == False) and 
                                                         ((indicator.name in filter_list) and (indicator.is_support_indicator == False) ) )   ]
    else:
        return [indicator for indicator in indicators if ((indicator.name in filter_list)  )   ]

def plot_candle_indicators(df, indicators,coin, step_size=10):

    part_no = 1
    for i in range(0,len(indicators),step_size):
        indicators_step = indicators[i:i+step_size]
        ap0 = []
        start=2
        panel_ratios_list=[]
        panel_ratios_list.append(4)
        panel_ratios_list.append(2)

        for indicator in indicators_step:

            ap0.append(mpf.make_addplot(df[indicator.name], color=indicator.color, panel=start, type=indicator.chart, ylabel="_".join(indicator.name.split("_")[1:])))
            ap0.append(mpf.make_addplot(df.VWAP,panel=0,color='fuchsia',secondary_y=True))
            #plot limits 
            if str(indicator.lower_limit) != "nan":
                lower_limit= df[indicator.name].copy(deep=True)
                lower_limit.values[:] = indicator.lower_limit
                ap0.append(mpf.make_addplot(lower_limit, color="r", panel=start, type="line"))

                
            if str(indicator.upper_limit) != "nan":
                upper_limit= df[indicator.name].copy(deep=True)
                upper_limit.values[:] = indicator.upper_limit
                ap0.append(mpf.make_addplot(upper_limit, color="r", panel=start, type="line"))


            for support_indicator in indicator.support_indicators:
                #print(support_indicator.color)
                ap0.append(mpf.make_addplot(df[support_indicator.name], color=support_indicator.color, panel=start, secondary_y=support_indicator.secondary_y, type=support_indicator.chart, ylabel="_".join(support_indicator.name.split("_")[1:])))
                
            start+=1
            panel_ratios_list.append(2)
        #print("panels ratios: ", panel_ratios_list)
        mpf.plot(df, type='candle', figscale=3.2, style='nightclouds', title=coin+"- part "+str(part_no),addplot=ap0,  volume=True,main_panel=0,volume_panel=1)#, panel_ratios=tuple(panel_ratios_list))
        part_no+=1

# Load dataset

In [ ]:
data_folder = "../input/g-research-crypto-forecasting/"
set_parameter_csv = {'sep': ',', 'encoding': 'ISO-8859-1', 'low_memory': True}
crypto_df = pd.read_csv('{}'.format(data_folder + 'train.csv'),      **set_parameter_csv)
crypto_df = reduce_mem_usage(crypto_df)
asset_details = pd.read_csv(data_folder + 'asset_details.csv')
crypto_df = crypto_df.merge(asset_details, how='left', left_on="Asset_ID", right_on="Asset_ID").copy(deep=True)
asset_names_dict = {row["Asset_ID"]:row["Asset_Name"] for ind, row in asset_details.iterrows()}
crypto_df["date"] = crypto_df["timestamp"].astype('datetime64[s]')
crypto_df.set_index("date", inplace=True)   # Set index to timestamp    

display(crypto_df.head(3))


# Data Dictionary

<div class="alert alert-block alert-warning">
<b>References:</b>
<ul>
    <li><a href = "https://www.kaggle.com/c/g-research-crypto-forecasting/data">Crypto-forecasting data dictionary</a></li>
</ul>
</div>

# Data Profiling

In [ ]:
profile_crypto = ProfileReport(crypto_df, title="Crypto -  Data Integrity & Profiling Report_df" , minimal=True)
profile_crypto.to_notebook_iframe()

# Nulls

In [ ]:
search_null(crypto_df).sort_values(by='% Total', ascending=False)

# Data Exploration

## Mean Close Price by Asset

BTC is by far the best valued currency


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))

data_agg=crypto_df[["Asset_Name","Close"]].groupby("Asset_Name")["Close"].agg(["mean"])
data_agg=data_agg.sort_values(by='mean',ascending=False)
data_agg.plot(kind="bar", ax=ax, color="blue", title="Mean Close Price by Asset")

## Boxplot by Asset and Year

We can see a high pump in 2020-2021 in all assets

In [ ]:

start = 0

fig, ax = plt.subplots(nrows=5, ncols=3, figsize=(30, 25))

for j in range(5):
    for i in range(3):
        if start == len(asset_names_dict):
            break 
        crypto_df_filtered = crypto_df[crypto_df["Asset_ID"]==start]
        crypto_df_filtered = crypto_df_filtered["Close"].groupby(pd.Grouper(freq='A'))
        years = pd.DataFrame()

        for name, group in crypto_df_filtered:
            years[name.year] = pd.Series( group.values )

        g= sns.boxplot( data=years, ax=ax[j,i])
        g.set_title(asset_names_dict[start])    
        start += 1
fig.tight_layout(pad=0.2)

## BTC VS ALT Coins

It seems BTC has been the pump driver over  years , as a result we can see how the other assets follow the btc trend.

In [ ]:
groups_assets =  crypto_df[["Asset_Name","Close"]].groupby([pd.Grouper("Asset_Name"),pd.Grouper(freq='M')]).max().reset_index()
btc = groups_assets[groups_assets["Asset_Name"]=="Bitcoin"]   # BTC   
lowprice_assets = ['Stellar','TRON', 'Cardano', 'IOTA', 'Dogecoin','EOS.IO']
others =  groups_assets[(~groups_assets["Asset_Name"].isin(lowprice_assets)) & (groups_assets["Asset_Name"]!="Bitcoin")]   # Others
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(25, 10))
ax2 = ax.twinx()
palette ={"Bitcoin": "#000000"}
g=sns.lineplot(data=btc, x="date", y="Close", hue="Asset_Name",err_style="bars", ci=68,ax=ax, palette=palette)
sns.move_legend(g, "upper left")
g=sns.lineplot(data=others, x="date", y="Close", hue="Asset_Name",err_style="bars", ci=68,ax=ax2)

In [ ]:
lowprice_assets = ['Cardano', 'IOTA', 'EOS.IO']
others =  groups_assets[(groups_assets["Asset_Name"].isin(lowprice_assets)) & (groups_assets["Asset_Name"]!="Bitcoin")]   # Others
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(25, 10))
ax2 = ax.twinx()
palette ={"Bitcoin": "#000000"}
g=sns.lineplot(data=btc, x="date", y="Close", hue="Asset_Name",err_style="bars", ci=68,ax=ax, palette=palette)
sns.move_legend(g, "upper left")
g=sns.lineplot(data=others, x="date", y="Close", hue="Asset_Name",err_style="bars", ci=68,ax=ax2)

In [ ]:
lowprice_assets = ['Stellar','TRON','Dogecoin']
others =  groups_assets[(groups_assets["Asset_Name"].isin(lowprice_assets)) & (groups_assets["Asset_Name"]!="Bitcoin")]   # Others
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(25, 10))
ax2 = ax.twinx()
palette ={"Bitcoin": "#000000"}
g=sns.lineplot(data=btc, x="date", y="Close", hue="Asset_Name",err_style="bars", ci=68,ax=ax, palette=palette)
sns.move_legend(g, "upper left")
g=sns.lineplot(data=others, x="date", y="Close", hue="Asset_Name",err_style="bars", ci=68,ax=ax2)

## Volatility

Volatility often refers to the amount of uncertainty or risk related to the size of changes in a security's value. A higher volatility means that a security's value can potentially be spread out over a larger range of values. This means that the price of the security can change dramatically over a short time period in either direction. A lower volatility means that a security's value does not fluctuate dramatically, and tends to be more steady.

<div class="alert alert-block alert-warning">
<b>References:</b>
<ul>
    <li><a href = "https://www.investopedia.com/terms/v/volatility.asp">Volatility</a></li>
</ul>
</div>

In [ ]:
btc = crypto_df[crypto_df["Asset_ID"]==1]
btc_grouped_1d = group_trading_timeseries(btc, time_step_size='1d')  
btc_grouped_1d = btc_grouped_1d.loc[pd.to_datetime('01/01/2021', format="%d/%m/%Y"):
                        pd.to_datetime('22/10/2021', format="%d/%m/%Y")]


mpf.plot(btc_grouped_1d, type='candle', figratio=(3,1), style='nightclouds', title="BTC Daily Price", volume=True)

Since the data is at one minute its a good idea to resample it in order to view in others timeframes, functions <b>group_trading_timeseries</b> and <b>group_trading_timeseries_all</b> are created for this purpose

## Technical Analysis

 <center><img src="https://errante.com/wp-content/uploads/2020/07/Errante-Trading-Tools-Understanding-Technical-Indicators.jpg" style = "max-height:300px;"></center> 
 
Technical analysis is a trading discipline employed to evaluate investments and identify trading opportunities by analyzing statistical trends gathered from trading activity, such as price movement and volume.

<div class="alert alert-block alert-warning">
<b>Key Takeaways:</b>
<ul>
    <li>Technical analysis is a trading discipline employed to evaluate investments and identify trading opportunities in price trends and patterns seen on charts.</li>
    <li>Technical analysts believe past trading activity and price changes of a security can be valuable indicators of the security's future price movements.</li>
    <li>Technical analysis is a trading discipline employed to evaluate investments and identify trading opportunities in price trends and patterns seen on charts.</li>
</ul>
<br/>



## Technical Analysis charts
Across the industry, there are hundreds of patterns and signals that have been developed by researchers to support technical analysis trading, it would be a bit tedious to code them. 
Fortunately there is a technical analysis library in python available(created by <a href="https://github.com/bukosabino" >bukosabino</a>), the library has implemented 42 indicators:


<b>Volume:</b>
<ul>
<li>Money Flow Index (MFI)</li>
<li>Accumulation/Distribution Index (ADI)</li>
<li>On-Balance Volume (OBV)</li>
<li>Chaikin Money Flow (CMF)</li>
<li>Force Index (FI)</li>
<li>Ease of Movement (EoM, EMV)</li>
<li>Volume-price Trend (VPT)</li>
<li>Negative Volume Index (NVI)</li>
<li>Volume Weighted Average Price (VWAP)</li>
</ul>
<b>Volatility:</b>
<ul>
<li>Average True Range (ATR)</li>
<li>Bollinger Bands (BB)</li>
<li>Keltner Channel (KC)</li>
<li>Donchian Channel (DC)</li>
<li>Ulcer Index (UI)</li>
</ul>

<b>Trend:</b>
<ul>
<li>Simple Moving Average (SMA)</li>
<li>Exponential Moving Average (EMA)</li>
<li>Weighted Moving Average (WMA)</li>
<li>Moving Average Convergence Divergence (MACD)</li>
<li>Average Directional Movement Index (ADX)</li>
<li>Vortex Indicator (VI)</li>
<li>Trix (TRIX)</li>
<li>Mass Index (MI)</li>
<li>Commodity Channel Index (CCI)</li>
<li>Detrended Price Oscillator (DPO)</li>
<li>KST Oscillator (KST)</li>
<li>Ichimoku Kinkō Hyō (Ichimoku)</li>
<li>Parabolic Stop And Reverse (Parabolic SAR)</li>
<li>Schaff Trend Cycle (STC)</li>
</ul>

<b>Momentum:</b>
<ul>
<li>Relative Strength Index (RSI)</li>
<li>Stochastic RSI (SRSI)</li>
<li>True strength index (TSI)</li>
<li>Ultimate Oscillator (UO)</li>
<li>Stochastic Oscillator (SR)</li>
<li>Williams %R (WR)</li>
<li>Awesome Oscillator (AO)</li>
<li>Kaufman's Adaptive Moving Average (KAMA)</li>
<li>Rate of Change (ROC)</li>
<li>Percentage Price Oscillator (PPO)</li>
<li>Percentage Volume Oscillator (PVO)</li>
</ul>
<b>Others:</b>
<ul>
<li>Daily Return (DR)</li>
<li>Daily Log Return (DLR)</li>
<li>Cumulative Return (CR)</li>
</ul>



<b>Documentation:</b>
<ul>
    <li><a href = "https://technical-analysis-library-in-python.readthedocs.io/en/latest/">Technical Analysis library documentation</a></li>
</ul>
</div>


In [ ]:
crypto_df_filtered = crypto_df[pd.to_datetime('01/01/2018', format="%d/%m/%Y"):
                        pd.to_datetime('02/01/2018', format="%d/%m/%Y")]
asset_list = [0,6] #Binance Coin, Ethereum
crypto_assets = crypto_df_filtered[crypto_df_filtered["Asset_ID"].isin(asset_list)]
crypto_assets= group_trading_timeseries_all(crypto_assets, time_step_size='5min')  
crypto_assets = add_all_ta_features(crypto_assets, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
crypto_assets = crypto_assets.reset_index()
crypto_assets.set_index("date", inplace=True)

In [ ]:
class TAIndicator():
    def __init__(self,  name="", type="", lower_limit=np.nan, upper_limit=np.nan, color='#00FF00', chart="line", support_indicators=[], is_support_indicator=False,secondary_y=False):
        self.name = name
        self.type = type
        self.lower_limit = lower_limit
        self.upper_limit = upper_limit
        self.color =  color
        self.color_function =  None

        self.chart = chart
        self.support_indicators = support_indicators
        self.is_support_indicator = is_support_indicator
        self.secondary_y = secondary_y
      
      

columns= crypto_assets.columns.values
indicators = []

for column in columns:
    if column.startswith("momentum"):
        indicators.append(TAIndicator(name=column, type="momentum"))
    elif column.startswith("volume") and column != "volume":
        indicators.append(TAIndicator(name=column, type="volume"))
    elif column.startswith("volatility"):
        indicators.append(TAIndicator(name=column, type="volatility"))
    elif column.startswith("trend"):
        indicators.append(TAIndicator(name=column, type="trend"))
    elif column.startswith("others"):
        indicators.append(TAIndicator(name=column, type="others"))

update_indicator_property(indicators, "momentum_rsi", "lower_limit", 30)
update_indicator_property(indicators, "momentum_rsi", "upper_limit", 70)
update_indicator_property(indicators, "momentum_stoch_rsi", "lower_limit", 0.3)
update_indicator_property(indicators, "momentum_stoch_rsi", "upper_limit", 0.7)

#support_indicators

#trend_macd', 'trend_macd_signal',     'trend_macd_diff'
#MACD
update_indicator_property(indicators, "trend_macd_diff", "chart", "bar")


#update_indicator_property(indicators, "trend_macd_diff", "color", macd_hist_color)


update_indicator_property(indicators, "trend_macd_signal", "is_support_indicator", True)#color
update_indicator_property(indicators, "trend_macd_signal", "color", "#FF0000")

update_indicator_property(indicators, "trend_macd_diff", "is_support_indicator", True)

#Bollinger bands
update_indicator_property(indicators, "volatility_bbl", "is_support_indicator", True)
update_indicator_property(indicators, "volatility_bbl", "secondary_y", False)
update_indicator_property(indicators, "volatility_bbw", "is_support_indicator", True)
update_indicator_property(indicators, "volatility_bbw", "secondary_y", False)
bb_support_ind= filter_indicators(indicators,['volatility_bbl'])
update_indicator_property(indicators, "volatility_bbh", "support_indicators", bb_support_ind)


macd_support_ind= filter_indicators(indicators,['trend_macd_signal', 'trend_macd_diff'])
update_indicator_property(indicators, "trend_macd", "support_indicators", macd_support_ind)




In [ ]:


indicators_plotted= ["momentum_rsi","momentum_stoch_rsi"]
indicators_plotted= []
indicators_plotted=['trend_macd',"momentum_stoch_rsi","momentum_rsi","volatility_bbh"]
indicators_plotted= [] #all

type_indicators=["momentum","volume","volatility","trend","others"]
#type_indicators=["momentum"]

#type_indicators=["volatility"]

for type in type_indicators:
    for moneda in asset_list:
        asset = crypto_assets[crypto_assets["Asset_ID"]==moneda]
        plot_candle_indicators(asset,filter_indicators(indicators,indicators_plotted,type),asset_names_dict[moneda]+"-"+type)


<h1>Continue....</h1>